# Visualizing Example Cells

In [74]:
import os
from meshparty import trimesh_io, trimesh_vtk, meshwork
from caveclient import CAVEclient
import numpy as np
import pandas as pd
import cloudvolume
import itkwidgets

#setting up the cave client, if you have not done this before checkout the notebook CAVEsetup.ipynb at 
#https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/CAVEsetup.ipynb
#client = CAVEclient()
#client.auth.get_new_token()
#client.auth.save_token(token="Your token here")

"""
dataset_name = 'minnie65_public_v117'
client = CAVEclient(dataset_name)
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)
# to enable to cache the mesh data, create a MeshMeta object:
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_v117_meshes',
                         map_gs_to_https=True)
"""

cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)

## Find matching elements across different databases

*sometimes the same cell can correspond to different `root_id` on different databases*

1. Find the soma coordinates of the element of interest (e.g `864691135687245024`)
    ```
    import pandas
    em_data = pandas.read_csv('../data/em_connectomics_mesh_skeletons_directory_proof_status.csv')
    ID = 864691135687245024
    em_data['pt_position'][em_data['root_id']==ID]
    ```
2. go to neuroglancer: https://ngl.microns-explorer.org

    - go to that position (top left coordinates)
    - find the other `root_id` that corresponds to that object
   
3. Download the associated mesh from the precomputed mesh database:
    ```
    MC_ID = 864691135367179769
    cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)
    mesh = cv.mesh.get(MC_ID,lod=3)[MC_ID]
    ```


## Showing both the Basket and Martinotti cells in the Cortical Column

In [72]:
MC_ID = 864691135367179769
mesh1 = cv.mesh.get(MC_ID,lod=3)[MC_ID]
poly_data1 = trimesh_vtk.trimesh_to_vtk(mesh1.vertices, mesh1.faces, None)

BC_ID = 864691135269913253
mesh2 = cv.mesh.get(BC_ID,lod=3)[BC_ID]
poly_data2 = trimesh_vtk.trimesh_to_vtk(mesh2.vertices, mesh2.faces, None)

L5P_ID = 864691135627205572
mesh3 = cv.mesh.get(L5P_ID,lod=3)[L5P_ID]
poly_data3 = trimesh_vtk.trimesh_to_vtk(mesh3.vertices, mesh3.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data2, poly_data1, poly_data3])

viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Basket Cell example

In [3]:
#selected one:
BC_ID = 864691135269913253
# see the Plot_Morphologies notebook to see them all
mesh = mm.mesh(seg_id=BC_ID) # and download it / just load it

In [8]:
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

### with synapses

In [75]:
cellid = BC_ID
mesh = cv.mesh.get(cellid,lod=3)[cellid]

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerBC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['g'])
viewerBC

AttributeError: 'CAVEclientGlobal' object has no attribute 'materialize'

# Martinotti Cell example

In [5]:
#MC_ID = 864691135467660940

In [ ]:
MC_ID = 864691137053906294 # L23 many synapses
meshw

In [33]:
MC_ID = 864691135367179769
mesh1 = cv.mesh.get(MC_ID,lod=3)[MC_ID]
poly_data1 = trimesh_vtk.trimesh_to_vtk(mesh1.vertices, mesh1.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data1])

viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [7]:
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [10]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Showing both together

In [12]:
mesh

Mesh(vertices<123839>, faces<216766>, normals<0>, segid=None, encoding_type=<draco>)

In [4]:
cellid = 864691135367179769
mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])

In [5]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Basket cell with synapses

In [6]:
cellid = BC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerBC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerBC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [27]:
cellid = MC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerMC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerMC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## More controlled rendering with `trimesh_vtk.render_actor`

In [ ]:
cellid = MC_ID

# mesh
mesh = cv.mesh.get(cellid,lod=3)[cellid]
mesh_actors = trimesh_vtk.mesh_actor(mesh, color=(0, 0, 0.4))
voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm

# synapses
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
syn_actors = trimesh_vtk.point_cloud_actor(syn_pts, size=syn_sizes/20, color=(1, 0, 0))
#                                           color=[() for i in range(len(syn_pts))])

#camera = trimesh_vtk.camera_from_quat()
trimesh_vtk.render_actors([syn_actors, mesh_actors])

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
ERROR:root:Cannot create GLX context.  Aborting.


In [69]:
trimesh_vtk.camera_from_quat?